<i><h2 style="font-family:serif;font-size:250%; text-align:center;color:#004d80"> Tarea III BDA: Neo4j</h2></i>
<i><h3 style="font-family:serif;font-size:100%; text-align:center;color:#FFFFF"> Jorge Caullán </h3></i></i>
<i><h3 style="font-family:serif;font-size:100%; text-align:center;color:#FFFFF"> Miguel Huichaman </h3></i></i>
<i><h3 style="font-family:serif;font-size:100%; text-align:center;color:#FFFFF"> Ignacio Loayza</h3></i></i>

# Introducción

A continuación se presenta el análisis de dos conjuntos de datos almacenados en el motor de base de datos `Neo4j`.
Los dos conjuntos a analizar corresponden a la red del metro de Santiago de Chile, donde los nodos corresponden a las estacione y las relaciones entre nodos son hechas en base a la conexión directa entras las estaciones.



### inicialización de la base de datos
nota: si se tiene problemas para conectar a neo4j desde el navegador, probar
* host: bolt://localhost:7687
* usuario: neo4j
* password: neo4j

1) si funciona, pedira cambiar el pass: poner cualquier cosa y usar como comando en el navegador
    * :server change-password
    * despues volver al password neo4j
   
2) si hay problemas, usar desde la consola donde esta el docker:
    * docker-compose exec neo4j bash
    * neo4j-admin set-initial-password neo4j

    y reintentar lo anterior

librerias requeridas:
* pip install py2neo

In [4]:
from pprint import pprint
from py2neo import Graph, Node, Relationship, NodeMatcher

# conectarse
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j"))

# graph.delete_all()

## I) Enrutamiento

> ### Ingestión de los datos hacia la base de datos

In [5]:
import json
with open('Santiago_de_Chile.json', encoding='utf-8') as fh:
    metros = json.load(fh)

matcher = NodeMatcher(graph)

for metro in metros:
    current_station_node = matcher.match("Station", name=metro['station']).first()
    if current_station_node:
        if current_station_node['line'] != metro['line']:
            current_station_node['line'] = '{},{}'.format(current_station_node['line'], metro['line'])
            graph.push(current_station_node)
    else:
        current_station_node = Node("Station", name=metro['station'], line=metro['line'])
        graph.create(current_station_node)

    if metro['previous'] != None:
        previous_node = matcher.match("Station", name=metro['previous']['station']).first()
        graph.create(Relationship(previous_node, "CONNECT", current_station_node, distance=metro['previous']['distance_in_meters']))
    if metro['next'] != None:
        next_node = matcher.match("Station", name=metro['next']['station']).first()
        if next_node:
            graph.create(Relationship(current_station_node, "CONNECT", next_node, distance=metro['next']['distance_in_meters']))

> ### 1) Seleccione y muestre todas las estaciones correspondientes a una lı́nea de metro (Excluyendo lı́nea 4A).

En este caso, recuperaremos todas las estaciones correspondientes a la linea `L4` que además no estén en la línea ´L4A´:

In [6]:
required_line = 'L4'

query = """
MATCH (n)
WHERE n.line contains {required_line} AND (not(n.line contains 'L4A'))
RETURN n
"""

data = graph.run(query, required_line=required_line)
print('Estaciones en {} y no en L4:\n'.format(required_line))
for d in data:
    print(dict(d)['n']['name'])

Estaciones en L4 y no en L4:

Tobalaba
Los Orientales
Grecia
Los Presidentes
Quilín
Las Torres
Macul
Vicente Valdés
Rojas Magallanes
Trinidad
San José de la Estrella
Los Quillayes
Elisa Correa
Plaza Egaña
Cristóbal Colón
Francisco Bilbao
Príncipe de Gales
Simón Bolívar
Las Mercedes
Plaza de Puente Alto
Hospital Sótero del Río
Protectora de la Infancia


<img src='estaciones L4 sin L4A.png'>

> ### 2) Muestre las 10 estaciones mas cercanas entre sí

Primero, se recuperan todos aquellos objetos que conectan con una estación, luego, se filtra solo aquellos atributos relevantes, en nuestro caso, `name` de la primera estación, `distance` que representa el peso de la arista entre las dos estaciones y `name` de la segunda estación. Finalmente, se ordena según `distance` (orden creciente) y se limita el retorno de la consulta a solo 10 elementos:

In [7]:
query = """
MATCH(first)-[c:CONNECT]->(second:Station)
RETURN first.name, c.distance, second.name 
ORDER BY c.distance 
LIMIT 10
"""

data = graph.run(query)

print('Las estaciones más cercanas entre sí son:\n')
for i, d in enumerate(data):
    data_dict = dict(d)
    print('{3}. {0} y {1}, separadas por un distancia de {2} [m]'.format(data_dict['first.name'],
                                                                data_dict['second.name'],
                                                                data_dict['c.distance'],
                                                                i+1))

Las estaciones más cercanas entre sí son:

1. La Moneda y Universidad de Chile, separadas por un distancia de 400 [m]
2. Los Héroes y La Moneda, separadas por un distancia de 460 [m]
3. Universidad de Chile y Santa Lucía, separadas por un distancia de 480 [m]
4. Parque Bustamante y Santa Isabel, separadas por un distancia de 500 [m]
5. Las Rejas y Ecuador, separadas por un distancia de 520 [m]
6. Santa Lucía y Universidad Católica, separadas por un distancia de 530 [m]
7. San Alberto Hurtado y Universidad de Santiago, separadas por un distancia de 540 [m]
8. Carlos Valdovinos y Camino Agrícola, separadas por un distancia de 540 [m]
9. Pedro de Valdivia y Los Leones, separadas por un distancia de 550 [m]
10. Unión Latinoamericana y República, separadas por un distancia de 560 [m]


<img src='10 estaciones mas cercanas.png'>

> ### 3) Muestre todas las estaciones con combinación.

In [12]:
query = """
MATCH (n:Station)
WHERE NOT(n.line in ['L1', 'L2', 'L3', 'L4', 'L4A', 'L5', 'L6'])
RETURN n
"""

data = graph.run(query)

for d in data:
    print(dict(d)['n']['name'])

San Pablo
Baquedano
Los Héroes
Universidad de Chile
Los Leones
Tobalaba
Puente Cal y Canto
Santa Ana
Franklin
La Cisterna
Irarrázaval
Ñuñoa
Plaza de Armas
Vicuña Mackenna
Vicente Valdés
Plaza Egaña
Ñuble


<img src='combinaciones.png'>

> ### 4) Busque la ruta mas corta entre una estacion de metro y la estacion Camino Agricola, esta ruta debe tener al menos 10 nodos y una combinacion

Utilizaremos como estación base la estación **_"Manuel Montt"_**. Además, pediremos las 2 rutas más cortas inmediatamente. En el caso de la ruta más corta:

In [13]:
query = """
MATCH (start:Station{name:"Manuel Montt"}), (end:Station{name:"Camino Agrícola"})
CALL algo.kShortestPaths.stream(start, end, 2, "distance")
YIELD index, nodeIds, costs
RETURN [node in algo.getNodesById(nodeIds) | node.name] AS places,
       costs,
       reduce(acc = 0.0, cost in costs | acc + cost) AS totalCost
"""

data_shortest, data_2_shortest = graph.run(query)

for i, d in enumerate(dict(data_shortest)['places']):
    print('{0}º {1}'.format(i+1, d))
print('\nLargo total de la ruta más corta (Manuel Montt -> Camino Agricola): {}[m]'.format(
                                                                            data_shortest['totalCost']))


1º Manuel Montt
2º Salvador
3º Baquedano
4º Parque Bustamante
5º Santa Isabel
6º Irarrázaval
7º Ñuble
8º Rodrigo de Araya
9º Carlos Valdovinos
10º Camino Agrícola

Largo total de la ruta más corta (Manuel Montt -> Camino Agricola): 8050.0[m]


> ### 5) Considere que ya no puede realizar esa combinacion, busque la siguiente ruta mas corta
La segunda ruta más corta es:

In [14]:
for i, d in enumerate(dict(data_2_shortest)['places']):
    print('{0}º {1}'.format(i+1, d))
print('\nLargo total de la segunda ruta más corta (Manuel Montt -> Camino Agricola): {}[m]'.format(
                                                                                    data_2_shortest['totalCost']))


1º Manuel Montt
2º Pedro de Valdivia
3º Los Leones
4º Inés de Suárez
5º Ñuñoa
6º Estadio Nacional
7º Ñuble
8º Rodrigo de Araya
9º Carlos Valdovinos
10º Camino Agrícola

Largo total de la segunda ruta más corta (Manuel Montt -> Camino Agricola): 11050.0[m]


## II) Centralidad y Comunidades
> ### Ingestión de los datos hacia la base de datos

Debido a la gran cantidad de datos que posee cada uno de los datasets, se opta por utilizar unicamente 10000 datos de cada uno ya que de lo contrario, la demora para agregar los datos era demaciada.

Habiendo definido esto, el codigo que se utiliza para agregar los datos es el siguiente

In [18]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j"))
# graph.delete_all()

matcher = NodeMatcher(graph)
c=0

# f = open('../soc-redditHyperlinks-title.tsv', 'r')
f = open('soc-redditHyperlinks-body.tsv', 'r')
f.readline() #para saltar la primera linea
all_lines = f.readlines()
f.close()

# total_lines = len(all_lines)
total_lines = 10000
post_ids = []

for row in all_lines:
    if c % (total_lines/100) == 0:
        print("progress:", c/float(total_lines)*100)
        
    #cada row es una referencia entre 2 subredits
    #el post id se puede repetir y las propeties son las mismas cuando se repiten
    row = row.replace('\n','').split('\t')
    if row[2] in post_ids:
        positive = 0
        negative = 0
    else:
        post_ids.append(row[2])
        if row[4] == '1':
            positive = 1
            negative = 0
        else:
            positive = 0
            negative = 1
    
    #crear subreddits si no existen
    subreddit_node = matcher.match("Subreddit", name=row[0]).first()
    if subreddit_node: #agregar el sentimiento del post al subredit
        subreddit_node['positives'] = subreddit_node['positives']+positive
        subreddit_node['negatives'] = subreddit_node['negatives']+negative
        graph.push(subreddit_node)
    else:
        subreddit_node = Node("Subreddit", name=row[0], positives=positive, negatives=negative)
        graph.create(subreddit_node)
        
    subreddit_hiperlink_node = matcher.match("Subreddit", name=row[1]).first()
    if not subreddit_hiperlink_node:
        subreddit_hiperlink_node = Node("Subreddit", name=row[1], positives=0, negatives=0)
        graph.create(subreddit_hiperlink_node)
    
    #relacion entre ambos subreddits
    graph.create(Relationship(subreddit_node, "REFERENCE", subreddit_hiperlink_node))
    
    c+=1
    if c==total_lines:
        break

progress: 0.0
progress: 1.0
progress: 2.0
progress: 3.0
progress: 4.0
progress: 5.0
progress: 6.0
progress: 7.000000000000001
progress: 8.0
progress: 9.0
progress: 10.0
progress: 11.0
progress: 12.0
progress: 13.0
progress: 14.000000000000002
progress: 15.0
progress: 16.0
progress: 17.0
progress: 18.0
progress: 19.0
progress: 20.0
progress: 21.0
progress: 22.0
progress: 23.0
progress: 24.0
progress: 25.0
progress: 26.0
progress: 27.0
progress: 28.000000000000004
progress: 28.999999999999996
progress: 30.0
progress: 31.0
progress: 32.0
progress: 33.0
progress: 34.0
progress: 35.0
progress: 36.0
progress: 37.0
progress: 38.0
progress: 39.0
progress: 40.0
progress: 41.0
progress: 42.0
progress: 43.0
progress: 44.0
progress: 45.0
progress: 46.0
progress: 47.0
progress: 48.0
progress: 49.0
progress: 50.0
progress: 51.0
progress: 52.0
progress: 53.0
progress: 54.0
progress: 55.00000000000001
progress: 56.00000000000001
progress: 56.99999999999999
progress: 57.99999999999999
progress: 59.0
pr

> ### 1) Detectar comunidades automáticamente, determine el tema de 3 de ellas y liste sus miembros.

Para esta pregunta, debido a la ambiguedad del concepto "comunidad", se utilizara como supuesto que una comunidad es un conjunto de subreddits que a su vez estan referenciados por al menos 5 otros subreddits.

A partir de esto, si utilizamos la siguiente consulta, podemos encontrar 3 comunidades y sus miembros:

In [16]:
query = """
MATCH (n:Subreddit)-[:REFERENCE]->(m)
WITH m, collect(n) as subsubreddits
WHERE size(subsubreddits) > 5
return m, subsubreddits limit 3
"""

data = graph.run(query)

comunidades = {}
for d in data:
    comunidades[d[0]['name']] = []
    for subsub in d[1]:
        comunidades[d[0]['name']].append(subsub['name'])
        
pprint(comunidades)

{'battlefield_4': ['xboxone',
                   'ps4',
                   'bestof',
                   'subredditdrama',
                   'gaming',
                   'bf4server',
                   'mwo',
                   'psbf',
                   'purebattlefield'],
 'games': ['battlefield_4',
           'bestof',
           'subredditdrama',
           'circlebroke2',
           'shitstatistssay',
           'starcraft',
           'gaming',
           'shitredditsays',
           'pcmasterrace',
           'truestl',
           'worstof',
           'srsgaming',
           'steam',
           'elderscrollsonline',
           'fallout',
           'eve',
           'playrust',
           'cynicalbrit',
           'credditgaming',
           'evedreddit',
           'overgrowth',
           'stargate',
           'starcraft2arcade'],
 'ps4': ['xboxone',
         'quityourbullshit',
         'shitredditsays',
         'pcmasterrace',
         'firstworldanarchists',
         'vi

<img src='comunidades.png'>

<img src='louvain_1.png'>

Hay una gran comunidad centrada alrededor del hub _'Gaming'_, observandose dos sub-comunidades de _'ps4'_ y 'battlefield_4'. Además, utilizando el algoritmo de detección de comunidades de Louvaina que forma parte del repertorio de `Neo4j`, se observa en la segunda imagen dos comunidades más: 'Christianity' y 'AskHistorians'

> ### 2) Encontrar los subreddits mas referenciados dentro de 3 comunidades diferentes. Que tienen en comun?(centralidad).

In [24]:
query = """
MATCH (n:Subreddit)-[:REFERENCE]->(m) 
return m, collect(n) as subsubreddits
order by size(subsubreddits) desc limit 3
"""

data = graph.run(query)

subreddits = {}
for d in data:
    subreddits[d[0]['name']] = []
    for subsub in d[1]:
        subreddits[d[0]['name']].append(subsub)
pprint(subreddits)

{'askreddit': [{'name': 'battlefield_4',
                'negatives': 0,
                'positives': 5},
               (_149:Subreddit {name: 'depthhub', negatives: 8, positives: 53}),
               {'name': 'panichistory',
                'negatives': 6,
                'positives': 35},
               (_158:Subreddit {name: 'wtf', negatives: 1, positives: 7}),
               {'name': 'fitnesscirclejerk',
                'negatives': 19,
                'positives': 87},
               {'name': 'bestoftldr',
                'negatives': 8,
                'positives': 25},
               {'name': 'quityourbullshit',
                'negatives': 14,
                'positives': 22},
               (_175:Subreddit {name: 'funny', negatives: 1, positives: 19}),
               (_178:Subreddit {name: 'nofap', negatives: 0, positives: 11}),
               {'name': 'switcharoo',
                'negatives': 5,
                'positives': 115},
               (_183:Subreddit {name: 'bitco

<img src="subreddits mas referenciados.png">

> ### 3) Considerando el atributo POST_LABEL, estudiar un caso donde una comunidad se refiera negativamente de otra.

Utilizando la siguiente consulta, se pueden tener ejemplos de comunidades a las cuales se han referido más de manera negativa que positiva:

In [27]:
query = """
match (n:Subreddit)-[:REFERENCE]->(m)
where n.positives < n.negatives
return m, collect(n) as subr
order by size(subr) desc limit 3
"""

data = graph.run(query)

subreddit_names = []
for d in data:
    print(d)

<Record m=(_199:Subreddit {name: 'askreddit', negatives: 20, positives: 124}) subr=[(_756:Subreddit {name: 'muhfeelings', negatives: 3, positives: 2}), (_956:Subreddit {name: 'itcrowd', negatives: 1, positives: 0}), (_1229:Subreddit {name: 'trueatheism', negatives: 4, positives: 2}), (_1642:Subreddit {name: 'toasterrights', negatives: 2, positives: 0}), (_1810:Subreddit {name: 'divorce', negatives: 1, positives: 0}), (_1833:Subreddit {name: 'mdma', negatives: 1, positives: 0}), (_1986:Subreddit {name: 'mynextband', negatives: 3, positives: 2}), (_2406:Subreddit {name: 'twobestfriendsplay', negatives: 1, positives: 0}), (_2949:Subreddit {name: 'antimemewatch', negatives: 3, positives: 0}), (_3492:Subreddit {name: '19thworldproblems', negatives: 1, positives: 0}), (_4863:Subreddit {name: 'entwives', negatives: 2, positives: 1}), (_5292:Subreddit {name: 'rayromano', negatives: 1, positives: 0}), (_5492:Subreddit {name: 'singing', negatives: 1, positives: 0}), (_6314:Subreddit {name: 'mass

<img src='subredits negativos.png'>